<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" style="padding-right:10px;width:140px;float:left"></td>
<h2 style="white-space: nowrap">Image Processing Laboratory Notebooks</h2>
<hr style="clear:both">
<p style="font-size:0.85em; margin:2px; text-align:justify">
This Juypter notebook is part of a series of computer laboratories which are designed
to teach image-processing programming; they are running on the EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course <b>Image Processing I</b> 
(<a href="https://moodle.epfl.ch/course/view.php?id=522">MICRO-511</a>) taught by Prof. M. Unser and Prof. D. Van de Ville.
</p>
<p style="font-size:0.85em; margin:2px; text-align:justify">
The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the <a href="http://bigwww.epfl.ch/">Biomedical Imaging Group</a>. 
The distribution or the reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL 2021.
</p>
<p style="font-size:0.85em; margin:0px"><b>Authors</b>: 
    <a href="mailto:pol.delaguilapla@epfl.ch">Pol del Aguila Pla</a>, 
    <a href="mailto:kay.lachler@epfl.ch">Kay Lächler</a>,
    <a href="mailto:alejandro.nogueronaramburu@epfl.ch">Alejandro Noguerón Arámburu</a>, and
    <a href="mailto:daniel.sage@epfl.ch">Daniel Sage</a>.
</p>
<hr style="clear:both">
<h1>Lab 2.1: Filtering</h1>
<div style="background-color:#F0F0F0;padding:4px">
    <p style="margin:4px;"><b>Released</b>: Thursday November 11, 2021</p>
    <p style="margin:4px;"><b>Submission</b>: <span style="color:red">Friday November 19, 2021</span> (before 11:59PM) on <a href="https://moodle.epfl.ch/course/view.php?id=522">Moodle</a></p>
    <p style="margin:4px;"><b>Grade weigth</b>: Lab 2 (18 points), 10% of the overall grade</p>
    <p style="margin:4px;"><b>Help sessions</b>: Monday November 15, on Zoom (12h-13h, see Moodle for link) and Thursday November 18 on campus</p>        
    <p style="margin:4px;"><b>Related lectures</b>: Chapter 3</p>
</div>

### Student Name: DAI Xianjie
### SCIPER: 336675|

Double-click on this cell and fill your name and SCIPER number. Then, run the cell below to verify your identity in Noto and set the seed for random results.

In [3]:
%use sos
import getpass
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

SCIPER: 336675


### Imports
In the next cell we import the Python libraries that we will use throughout the lab, as well as the `ImageViewer` class (Python package developed specifically for these laboratories, see documentation [here](https://github.com/Biomedical-Imaging-Group/interactive-kit/wiki/Image-Viewer), or run the python command `help(viewer)` after loading the class):

* [`matplotlib.pyplot`](https://matplotlib.org), to display images
* [`ipywidgets`](https://ipywidgets.readthedocs.io/en/latest/), to make the image display interactive
* [`numpy`](https://numpy.org/doc/stable/reference/index.html), for mathematical operations on arrays
* [`openCV` (cv2)](https://docs.opencv.org/2.4/index.html), for image-processing tasks
* [`scipy.ndimage`](https://docs.scipy.org/doc/scipy/reference/ndimage.html), Scipy's specific module for multidimensional image processing

Finally, we load the images you will use in the exercise to test your algorithms. 

In [4]:
%use sos
# Configure plotting as dynamic
%matplotlib widget

# Import standard required packages for this exercise
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv 
import scipy.ndimage as ndi
import ipywidgets as widgets
from interactive_kit import imviewer as viewer

# Load images to be used in this exercise 
bikesgray = cv.imread('images/bikesgray.tif', cv.IMREAD_UNCHANGED).astype('float64')
camera = cv.imread('images/camera-16bits.tif', cv.IMREAD_UNCHANGED).astype('float64')
spots = cv.imread('images/spots.tif', cv.IMREAD_UNCHANGED).astype('float64')

Now we will import the `ImageAccess` class. You can find the documentation of the class [here](https://biomedical-imaging-group.github.io/image-access/). Moreover, we will put the images we will be using in the JavaScript kernel.

In [5]:
%use javascript
%get bikesgray camera spots
// import ImageAccess as Image
var Image = require('image-access')

# Filtering (9 points)

In this lab you will review the basics of digital image filtering. You will review how to implement a convolution (the main ingredient of filtering), and you will review topics such as separable filters, non-separable filters, filter design and boundary conditions. The goal is for you to fully understand the filtering algorithm at a pixel-by-pixel level (implementing it in JavaScript), and at the same time to learn the standard tools to use filtering in professional image-processing libraries in Python.

## <a id="ToC_1_Filtering"></a>Table of contents
1. [Edge detection](#1.-Edge-detection-(3-points))
    1. [Non-separable implementation](#1.A.-Non-separable-implementation-(1-point)) (**1 point**)
    2. [Separable implementation](#1.B.-Separable-implementation-(2-points)) (**2 points**)
    3. [Comparison: Separable vs Non-separable](#1.C.-Comparison:-Separable-vs-Non-separable) 
    4. [Edge detection in Python](#1.D.-Edge-detection-in-Python)
2. [Implementation and classification of digital filters](#2.-Implementation-and-classification-of-digital-filters-(6-points)) (**6 points**)
    1. [Mask A](#2.A.-Mask-A)
    2. [Mask B](#2.B.-Mask-B)
    3. [Mask C](#2.C.-Mask-C)
    4. [Mask D](#2.D.-Mask-D)
    5. [Mask E](#2.E.-Mask-E)
    6. [Mask F](#2.F.-Mask-F)

<div class=" alert alert-danger">

<b>Important:</b> Each cell that contains code begins with `%use sos` or `%use javascript`. This indicates if the code in this specific cell should be written in Python or JavaScript. Do not change or remove any lines of code that begin with an %. They are used for the notebook to run smoothly with <code>SoS</code> and need to be on the first line of each cell!
</div>

Good luck and enjoy! 

### Visualize images
Get familiar now with the images you are going to be using. Remember to browse through the images with the buttons `Next` & `Prev`, and pay attention to the histograms. 

In [4]:
%use sos
# Declare image_list for ImageViewer
image_list = [bikesgray, camera, spots]

imgs_viewer = viewer(image_list, widgets=True, hist=True)

# 1. Edge detection (3 points)
[Back to table of contents](#ToC_1_Filtering)

In this part of the lab, we will review one of the most basic image processing operation, edge detection. Moreover, we will use this filter to review the concepts of convolution and correlation. As you might remember from the course, edge detection algorithms try to outline steep intesity differences between neighbouring pixels. A common vertical-edge detector (detecting changes in the $x$ direction) is the filter represented by the impulse response 

$$h[m,n] = 
\begin{bmatrix}
    -1 & 0 & 1 \\
    -2 & 0 & 2 \\
    -1 & 0 & 1
\end{bmatrix}\,.$$

If pixels neighbouring a given pixel position all have a similar value, the output at that pixel will have a value close to zero after applying the mask. On the other hand, if there is a big difference in the values of the left and right columns, this likely indicates some kind of vertical edge and consequently the output at that location will have either a highly positive or highly negative intensity.  

## 1.A. Non-separable implementation (1 point)
[Back to table of contents](#ToC_1_Filtering)

In the next cell we provide the function `filterNonSeparable()`. This function **is supposed to perform a convolution** on two input images `img` and `mask`. However, whoever implemented the function (*it certainly wasn't us*) had not heard about the difference between a correlation and a convolution. Long story short, the function does not provide the correct result. **For 1 point**, find the mistake and correct it in the cell below.

<div class="alert alert-success">

<b>Hints</b>: <ul><li>The two-dimensional convolution between two images <code>h</code> and <code>f</code> is given by
$$(h \ast f)[x,y] = \sum_{m \in \mathbb{Z}}\sum_{n \in \mathbb{Z}}f[m,n]h[x-m,y-n]$$</li></ul>
<li>There are a few considerations that you will need to keep in mind when going from formula to code. For example, in the formula above, $x-m$ is negative for some values, but happens if you try to acces the $-1$ index of a JavaScript Array?</li>
</div>

In [5]:
%use javascript

// function that performs a convolution on the two input parameters img and mask
function filterNonSeparable(img, mask){
    // create output Image, of the same shape as the input
    var output = new Image(img.shape());
    // iterate through every pixel
    for(var x = 0; x < img.nx; x++){  
        for(var y = 0; y < img.ny; y++){
            // get neighbourhood with the size of mask of current position
            var neigh = img.getNbh(x, y, mask.nx, mask.ny)
            // perform convolution
            var val = 0;
            for(var k = 0; k < mask.nx; k++){
                for(var l = 0; l < mask.ny; l++){
                    val += neigh.getPixel(k, l) * mask.getPixel(mask.nx-1-k, mask.ny-1-l);
                }
            }
            // assign value on output image
            output.setPixel(x, y, val)
        }
    }
    return output
}

Run the next cell to perform a sanity check (convolution of the mask with an impulse image).

In [6]:
// Lets do a sanity check
// we use a simple 3x3 impulse image
var impulse = new Image([[0, 0, 0], [0, 1, 0], [0, 0, 0]]);
// declare mask
var mask = new Image([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]]);
// check the result
console.log('Convolving by an impulse should return the unchanged mask.\nMask:\n' + mask.visualize() )
console.log('Result:\n' + filterNonSeparable(impulse, mask).visualize());
if(filterNonSeparable(impulse, mask).imageCompare(mask) == false){
    console.log('\nWARNING!!!\nThe function `filterNonSeparable` still provides an incorrect output.\n')
}else{
    console.log('It seems that the function performs a convolution.')
}

Convolving by an impulse should return the unchanged mask.
Mask:
[[ -1  0  1 ]
 [ -2  0  2 ]
 [ -1  0  1 ]]

Result:
[[ -1  0  1 ]
 [ -2  0  2 ]
 [ -1  0  1 ]]

It seems that the function performs a convolution.


### Applying the function

Run the next cell to apply the `filterNonSeparable` function with the mask given above to the images `bikesgray`, `camera` and `spots`. 

In [7]:
%use javascript
%get bikesgray camera spots

// apply Non separable filter
var vedge_bike_nons   = filterNonSeparable(new Image(bikesgray), mask).toArray();
var vedge_spots_nons  = filterNonSeparable(new Image(spots),     mask).toArray();
var vedge_camera_nons = filterNonSeparable(new Image(camera),    mask).toArray();

### Visualize the results
In the next cell we will visualize the results of edge detection. Run the next cell to visualize the results and use the `Next` / `Prev` buttons to cycle through the images.

In [8]:
%use sos
%get vedge_bike_nons vedge_camera_nons vedge_spots_nons --from javascript

# Declare parameters for viewer
image_list_nons = [vedge_bike_nons, vedge_camera_nons, vedge_spots_nons]
title_list = ['Bike - Vertical edges', 'Camera - Vertical edges', 'Spots - Vertical edges']

# Make sure that the images are numpy arrays and not lists
for image in range(len(image_list_nons)):
    image_list_nons[image] = np.array(image_list_nons[image])

vedge_nons_viewer = viewer(image_list_nons, title = title_list, widgets=True)

## 1.B. Separable implementation (2 points)
[Back to table of contents](#ToC_1_Filtering)

In this subsection, you are going to modify the function `filterSeparable()` we provide to implement the separable version of the filter. This function is based on the function `filter1D()` (also declared in the next cell), which takes as input a row/column of the image and a 1D mask and performs a 1D convolution. Recall that separable filters are divided into two 1D masks (`mx` and `my` in the code below). 

In the next cell, **for 1 point** complete the function `filterSeparable()` by implementing the column filtering. The row filtering is provided and can be used as a reference.

<div class="alert alert-warning">
    
<b>Technical Note:</b> <code>img.getRow(y)</code> extracts the $y^{\text{th}}$ row from <code>img</code> while <code>output.putRow(y, new_row)</code> inserts <code>new_row</code> into the $y^{\text{th}}$ row of <code>output</code>. Similarly you can use <code>.getColumn(x)</code> and <code>.putColumn(x, new_column)</code> to extract and insert columns from/to <code>Image</code> objects.
</div>
<div class="alert alert-info">
    
<b>Hint:</b> If you don't remember how a separable filter works, review the theory in your course notes.
</div>

In [29]:
%use javascript

// function that performs a separable convolution on img using two 1D masks mx and my
function filterSeparable(img, mx, my){
    // initialize output Image, of the same shape as the input
    var output = new Image(img.shape());
    
    // iterate through every row 
    for(var y = 0; y < img.ny; y++){
        // extract row
        var row = img.getRow(y);
        // apply filter
        var new_row = filter1D(row, mx);
        // set column in output variable
        output.putRow(y, new_row);    
    }
    // iterate through every column
    for(var x = 0; x < output.nx; x++){
        // extract row
        var col = output.getColumn(x);
        // apply filter
        var new_col = filter1D(col, my);
        // set column in output variable
        output.putColumn(x, new_col);   
        
    }
    return output
}

// function that applies a 1D filter
function filter1D(img, mask){
    // transpose the input variables if necessary
    if(img.nx == 1){
        img.transposeImage();
    }
    if(mask.nx == 1){
        mask.transposeImage();
    }
    // create the output image
    var output = new Image(img.shape());
    // iterate through all pixels
    for(var x = 0; x < img.nx; x++){
        // get the neighbourhood around position x
        var neigh = img.getNbh(x, 0, mask.nx, 1);
        // declare a variable to store the values of the convolution. 
        var val = 0;
        // iterate through the neighbourhood
        for(var i = 0; i < neigh.nx; i++){
            // perform convolution
            val += neigh.getPixel(i, 0) * mask.getPixel(mask.nx - 1 - i, 0);
        }
        // set value in output array
        output.setPixel(x, 0, val);
    }
    return output
}

### 1D masks

Now, before testing your separable implementation and **for 1 point** change the masks `mask_x` and `mask_y` in the cell below to the correct 1D masks corresponding to the separable version of the vertical-edge detection filter presented [at the beginning of this section](#1.-Edge-detection-(3-points)).

In [30]:
%use javascript
%put mask_x mask_y
// // change the masks to the correct value 
var mask_x = [-1, 0, 1];
var mask_y = [1, 2, 1];

// YOUR CODE HERE

// convert the masks to Image objects
var mask_x_img = new Image([mask_x]);
var mask_y_img = new Image([mask_y]);

Now, run the next cell for a quick sanity check on your function `filterSeparable` and on your $1$D masks. We will convolve an impulse with your $1$D masks, and the result should be exactly the filter that we presented [at the beginning of the section](#1.-Edge-detection-(3-points)).

In [31]:
%use javascript
// again let's do a sanity check with an impulse image
var impulse = new Image([[0, 0, 0], [0, 1, 0], [0, 0, 0]]);
// declare correct 2D mask
var mask = new Image([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]);
// the result of a convolution with an impulse image should be the mask itself
console.log('Convolving by an impulse should return the original 2D mask.\nMask:\n' + mask.visualize() )
console.log('Result:\n' + filterSeparable(impulse, mask_x_img, mask_y_img).visualize(), '\n');
if(filterSeparable(impulse, mask_x_img, mask_y_img).imageCompare(mask) == false){
    console.log('WARNING!!!\nSorry it looks like the separable filter or the 1D masks are not yet correct.');
}else{
    console.log('The separable filter and the 1D masks have passed this sanity check!');
}

Convolving by an impulse should return the original 2D mask.
Mask:
[[ -1  0  1 ]
 [ -2  0  2 ]
 [ -1  0  1 ]]

Result:
[[ -1  0  1 ]
 [ -2  0  2 ]
 [ -1  0  1 ]]
 

The separable filter and the 1D masks have passed this sanity check!


Now run the next cell to apply the function to the same three images we have been using before.

In [32]:
%use javascript
%get bikesgray spots camera
// Apply function on previously defined Images
var vedge_bike_s = filterSeparable(new Image(bikesgray), mask_x_img, mask_y_img).toArray();
var vedge_spots_s = filterSeparable(new Image(spots), mask_x_img, mask_y_img).toArray();
var vedge_camera_s = filterSeparable(new Image(camera), mask_x_img, mask_y_img).toArray();

Run the next one to visualize the results.

In [33]:
%use sos
%get vedge_bike_s vedge_camera_s vedge_spots_s --from javascript
# Declare the image list
image_list_sep = [vedge_bike_s, vedge_camera_s, vedge_spots_s]
title_list = ['Bike - Vertical edges', 'Camera - Vertical edges', 'Spots - Vertical edges']

# Make sure that the images are numpy arrays and not lists
for i, image in enumerate(image_list_sep):
    image_list_sep[i] = np.array(image)

plt.close('all')
vedge_s_viewer = viewer(image_list_sep, title = title_list, widgets=True)

## 1.C. Comparison: Separable vs Non-separable
[Back to table of contents](#ToC_1_Filtering)

Did you notice any difference between the separable and non-separable versions of the results? Did you expect any difference? 

Explore the two histograms, zoom into different areas of the three images, and see if they are truly equal. There are several tests you can do to measure the equality of both implementations. You can even take the difference between the two images and visualize it (`image1 - image2`), or you can use the different functions provided by NumPy to compare arrays (see [numpy.testing.assert_array_equal](https://numpy.org/doc/stable/reference/generated/numpy.testing.assert_array_equal.html) and [numpy.testing.assert_array_almost_equal](https://numpy.org/doc/stable/reference/generated/numpy.testing.assert_array_almost_equal.html)). Use the next blank cell to do any necessary exploration. 

<div class = 'alert alert-info'>
<b>Hint</b>: If both your implementations are correct, the difference should be negligible. 
</div>

In [61]:
%use sos

image_list_nons = [vedge_bike_nons, vedge_camera_nons, vedge_spots_nons]
image_list_sep = [vedge_bike_s, vedge_camera_s, vedge_spots_s]

np.testing.assert_array_equal(vedge_bike_nons, vedge_bike_s)
print("Images of Bike has no difference")
np.testing.assert_array_equal(vedge_camera_nons, vedge_camera_s)
print("Images of Camera has no difference")
np.testing.assert_array_equal(vedge_spots_nons, vedge_spots_s)
print("Images of Spots has no difference")


Images of Bike has no difference
Images of Camera has no difference
Images of Spots has no difference


## 1.D. Edge detection in Python
[Back to table of contents](#ToC_1_Filtering)

In Python, the two functions `filterNonSeparable` and `filterSeparable` you implemented above are provided by the [`scipy.ndimage`](https://docs.scipy.org/doc/scipy/reference/ndimage.html) library. They are called [`convolve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.convolve.html) and [`convolve1d`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.convolve1d.html), respectively (click on links to see the documentation). 

The arguments of this functions are:

 * `input`: the original image
 * `weights`: the mask to convolve the image with
 * `axis` **(only for `convolve1d`)**: The axis along which the 1D filter will be applied ($0$ to filter columns, $1$ for rows)
 * `output`: array or dtype (see technical note below)
 * `mode`: specifies the boundary conditions (see documentation for options). Defaults to `'reflect'`, which is the same we recommend and use in the course, as well as the one implemented in the JavaScript class `ImageAccess`.

**Examples:**

To apply a non-separable filter with a 2D mask `mask` to the image `img` we use:
```python
filtered_img = ndi.convolve(img, mask)
```
To apply a separable filter with two 1D masks `mask_x` and `mask_y` to the image `img` we use:
```python
# filter the columns
filtered_img = ndi.convolve1d(img, mask_y, axis=0)
# filter the rows
filtered_img = ndi.convolve1d(filtered_img, mask_x, axis=1)
```

<div class="alert alert-warning">

<b>Technical note:</b> 
In this lab we have been using <code>'float64'</code> ($64$-bit floating point) as the <code>dtype</code> (which stands for <i>data type</i>) of the images (look at the <a href="#Imports">Imports</a> section). This <code>dtype</code> is the most versatile, and since the <code>dtype</code> of the output array is the same (by default) as the dtype of the input array, we don't really need to mind the <code>output</code> parameter right now. However, it is also very common to store images and work with them as <code>np.uint8</code> (8-bit unsigned integer, which as you should know spans the range $[0, 255]$). Nonetheless we know that the gradient can take large positive and negative values, so an <a href="https://en.wikipedia.org/wiki/Integer_(computer_science)" >unsigned integer</a> is not the right type. If you don't realize why, do a quick experiment: declare a NumPy Array of length one, of data type <code>'uint8'</code>, and assign a value outside of the range $[0, 255]$ (e.g. <code>np.array([300], dtype = 'uint8')</code>). Perform a few operations and see what happens.
    
If at some point you need to work with integers (e.g. to save memory or to increase performance), because we know that the result will still be an integer -the mask only contains integers- you can choose <code>np.int32</code> (32-bit signed integers). 
    
This is something that you will have to keep in mind constantly when working with images, and that can be the source of many bugs.
</div>

Run the next cell to compare the Python implementations to the JavaScript implementations, and make sure that you get the same result. 
<div class="alert alert-info">
<b>Note:</b> If you did everything correctly in Sections <a href="#1.A.-Non-separable-implementation-(1-point)">1.A.</a> and <a href = '#1.B.-Separable-implementation-(2-points)'>1.B.</a>, this cell will display a winning message. Otherwise, it will plot your images <b>with the regions that differ from the correct answer highlighted in red</b>, this should give you a clue on where you might have gone wrong. For the purpose we will use the function <a href="https://numpy.org/doc/stable/reference/generated/numpy.allclose.html"><code>np.allclose</code></a>, a standard NumPy comparison tool.
</div>

In [62]:
%use sos 
# Define the 2D mask
mask = np.array([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]])
# Apply non-separable Filter
vedge_ndi = ndi.convolve(bikesgray, mask, output=np.int32)
# Apply separable Filter - we use mask_x and mask_y you defined in the exercise before
# filter the columns
vedge_ndi_s = ndi.convolve1d(bikesgray, mask_y, axis=0, output=np.int32)
# filter the rows
vedge_ndi_s = ndi.convolve1d(vedge_ndi_s, mask_x, axis=1, output=np.int32)

# Visualize if necessary
plt.close('all')
if np.allclose(vedge_ndi, vedge_bike_nons):
    print('Your non-separable implementation is correct!')
else :
    print('Review your non-separable JS implementation! \
           \nThe problem could be either on your 1D masks, or on the implementation itself. The previous sanity checks and the red areas should guide you.')
    viewer([np.array(vedge_bike_nons), vedge_ndi], title=['JS non-separable', 'Python non-separable'], compare=True)

if np.allclose(vedge_ndi_s, vedge_bike_s):
    print('Congratulations!\nYour separable filter is correct!')
else :
    print('Review your separable JS implementation.')
    viewer([np.array(vedge_bike_s), vedge_ndi_s], title=['JS separable', 'Python separable'], compare=True)

# vedge_viewer = viewer([np.array(vedge_bike_nons), vedge_ndi, np.array(vedge_bike_s), vedge_ndi_s], title=['JS non-separable', 'Python non-separable', 'JS separable', 'Python separable'], subplots=(2,2))

Your non-separable implementation is correct!
Congratulations!
Your separable filter is correct!


# 2. Implementation and classification of digital filters (6 points)
[Back to table of contents](#ToC_1_Filtering)

In this exercise, we will practice the implementation and classification of masks or digital filters. Moreover, you will practice separable and non-separable implementations in Python. We will look at $6$ different digital filters. Your job, **for 6 points**, is to implement and apply to an image each one of the filters in Python (**0.5** points per filter), and classify their visual effect (**0.5** points per filter).

Note that you can define a 2D mask in Python with 
```python
mask = np.array([[ 1,  2,  3,  4,  5], 
                 [ 6,  7,  8,  9, 10], 
                 [11, 12, 13, 14, 15],
                 [16, 17, 18, 19, 20],
                 [21, 22, 23, 24, 25]])
```
and define a 1D mask in Python with
```python
mask = np.array([1, 2, 3, 4, 5])
```

<div class="alert alert-danger">

<b>Important:</b> If a $2$D mask is separable, it is best practice to implement it as a separable filter using <code>ndi.convolve1d</code> with the appropriate 1D masks (that you will have to find). Otherwise, use <code>ndi.convolve</code>. Review Section [1.D.](#-1.D.-Edge-detection-in-Python) for an example. <b>If a mask is separable and you do not implement it as such, you will not get any points</b>. 
</div>

## 2.A. Mask A
[Back to table of contents](#ToC_1_Filtering)

In the cell below, **For 0.5 points**, implement the function `filter_A`, which convolves an image given as input parameter with the filter

```python3
    [ 0,  0,  1,  0,  0]
    [ 0,  1,  2,  1,  0]
A = [ 1,  2,-16,  2,  1]
    [ 0,  1,  2,  1,  0]
    [ 0,  0,  1,  0,  0]
```

Remember to check if the filter is separable, and in that case, implement it in that way.

<div class = 'alert alert-danger'>
<b>Note</b>: Hard-code the mask inside the function. That means, do not take the mask from a local variable, nor as input parameter of the function <code>filter_A</code> (the only input parameter is an image), but declare the mask <b>inside</b> the function. This is true for all the following 6 exercises.
</div>

In [1]:
%use sos

# Function that filters img with the mask A
def filter_A(img):
    output = img.copy()
    
    # Filter the image with mask A    
    mask_A = np.array([[ 0,  0,  1,  0,  0],
                       [ 0,  1,  2,  1,  0],
                       [ 1,  2,-16,  2,  1],
                       [ 0,  1,  2,  1,  0],
                       [ 0,  0,  1,  0,  0]])
    
    # Filter the image with mask A
    output = ndi.convolve(img, mask_A, output=np.int32)
    
    return output

Run the next cell to visualize the result of applying `filter_A` to the image `bikesgray`. 
<div class = "alert alert-info"><b>Note</b>: If you cannot see the whole <code>viewer</code>, remember that you can use <code>Ctrl</code>+<code>b</code> to hide the sidebar at the left. If you have already done this, you can always use <code>Ctrl</code> + <code>-</code> to zoom out a bit</div>

In [6]:
%use sos
# Apply the function
bikesgray_A = filter_A(bikesgray)
# Visualize the result
plt.close('all')
vis_A = viewer([bikesgray, bikesgray_A], title=['Original', 'Convolved with filter A'], subplots=(1,2))

Button(description='Show Widgets', style=ButtonStyle())

**For 0.5 points** answer the following MCQ:

* Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring, or
2. vertical blurring, or
3. horizontal blurring, or
4. diagonal blurring, or
5. sharpening, or
6. vertical-edge detection, or
7. horizontal-edge detection, or
8. diagonal edge detection, or
9. Laplacian: a second order derivative to find edges (maxima and minima), or
10. it has not effect.

In [7]:
%use sos
# Assign your answer to this variable
answer = 9
# YOUR CODE HERE

In [8]:
%use sos
# Sanity check
if not answer in list(range(1,11)):
    print('WARNING!\nAnswer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10.')

## 2.B. Mask B
[Back to table of contents](#ToC_1_Filtering)

In the cell below, **For 0.5 points**, implement the function `filter_B`, which convolves the input image with the filter

```python3
    [ 0,  1,  2,  1,  0]
    [ 1,  2,  4,  2,  1]
B = [ 2,  4,  8,  4,  2]
    [ 1,  2,  4,  2,  1]
    [ 0,  1,  2,  1,  0]
```

Remember to check if the filter is separable, and in that case, implement it in that way.

In [9]:
%use sos

# Function that filters img with the mask B
def filter_B(img):
    output = img.copy()
    
    # Filter the image with mask B    
    mask_B = np.array([[ 0,  1,  2,  1,  0],
                       [ 1,  2,  4,  2,  1],
                       [ 2,  4,  8,  4,  2],
                       [ 1,  2,  4,  2,  1],
                       [ 0,  1,  2,  1,  0]])
    
    # Filter the image with mask B    
    output = ndi.convolve(img, mask_B, output=np.int32)
    
    return output

Run the next cell to visualize the result of applying `filter_B` to the image `bikesgray`.

In [10]:
%use sos
# Apply the function
bikesgray_B = filter_B(bikesgray)
# Visualize the result
plt.close('all')
vis_B = viewer([bikesgray, bikesgray_B], title=['Original', 'Filtered with mask B'], subplots=(1,2))

Button(description='Show Widgets', style=ButtonStyle())

**For 0.5 points** answer the following MCQ:

* Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring, or
2. vertical blurring, or
3. horizontal blurring, or
4. diagonal blurring, or
5. sharpening, or
6. vertical-edge detection, or
7. horizontal-edge detection, or
8. diagonal edge detection, or
9. Laplacian: a second order derivative to find edges (maxima and minima), or
10. it has not effect.

In [ ]:
%use sos
# Assign your answer to this variable
answer = 1
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity check
if not answer in list(range(1,11)):
    print('WARNING!\nAnswer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10.')

## 2.C. Mask C
[Back to table of contents](#ToC_1_Filtering)

In the cell below, **For 0.5 points**, implement the function `filter_C`, which convolves the input image with the filter

```python3
    [-1,  -1,  -1,  -1,  -1]
    [-2,  -2,  -2,  -2,  -2]
C = [ 0,   0,   0,   0,   0]
    [ 2,   2,   2,   2,   2]
    [ 1,   1,   1,   1,   1]
```

Remember to check if the filter is separable, and in that case, implement it in that way.

In [38]:
%use sos

# Function that filters img with the mask C
def filter_C(img):
    output = img.copy()
    
    # Filter the image with mask C
    
    mask_x = np.array([1, 1, 1, 1, 1])
    mask_y = np.array([-1, -2, 0, 2, 1])
    
    # filter the columns
    filtered_img = ndi.convolve1d(output, mask_y, axis=0)
    # filter the rows
    filtered_img = ndi.convolve1d(filtered_img, mask_x, axis=1)
    return filtered_img

Run the next cell to visualize the result of applying `filter_C` to the image `bikesgray`.

In [39]:
%use sos
# Apply the function
bikesgray_C = filter_C(bikesgray)
# Visualize the result
plt.close('all')
vis_C = viewer([bikesgray, bikesgray_C], title=['Original', 'Filtered with mask C'], subplots=(1,2))

Button(description='Show Widgets', style=ButtonStyle())

**For 0.5 points** answer the following MCQ:

* Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring, or
2. vertical blurring, or
3. horizontal blurring, or
4. diagonal blurring, or
5. sharpening, or
6. vertical-edge detection, or
7. horizontal-edge detection, or
8. diagonal edge detection, or
9. Laplacian: a second order derivative to find edges (maxima and minima), or
10. it has not effect.

In [36]:
%use sos
# Assign your answer to this variable
answer = 7
# YOUR CODE HERE

In [37]:
%use sos
# Sanity check
if not answer in list(range(1,11)):
    print('WARNING!\nAnswer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10.')

## 2.D. Mask D
[Back to table of contents](#ToC_1_Filtering)

In the cell below, **For 0.5 points**, implement the function `filter_D`, which convolves the input image with the filter

```python3
    [-1,  -4,  -6,  -4, -1]
    [-4, -16, -24, -16, -4]
D = [-6, -24, 476, -24, -6]
    [-4, -16, -24, -16, -4]
    [-1,  -4,  -6,  -4, -1]
```

Remember to check if the filter is separable, and in that case, implement it in that way.

In [11]:
%use sos

# Function that filters img with the mask D
def filter_D(img):
    output = img.copy()
    
    # Filter the image with mask D
    
    mask_D = np.array([[-1,  -4,  -6,  -4, -1],
                       [-4, -16, -24, -16, -4],
                       [-6, -24, 476, -24, -6],
                       [-4, -16, -24, -16, -4],
                       [-1,  -4,  -6,  -4, -1]])
    
    # Filter the image with mask D
    output = ndi.convolve(output, mask_D, output=np.int32)
    
    return output

Run the next cell to visualize the result of applying `filter_D` to the image `bikesgray`.

In [12]:
%use sos
# Apply the function
bikesgray_D = filter_D(bikesgray)
# Visualize the result
plt.close('all')
vis_D = viewer([bikesgray, bikesgray_D], title=['Original', 'Filtered with mask D'], subplots=(1,2))

Button(description='Show Widgets', style=ButtonStyle())

**For 0.5 points** answer the following MCQ:

* Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring, or
2. vertical blurring, or
3. horizontal blurring, or
4. diagonal blurring, or
5. sharpening, or
6. vertical-edge detection, or
7. horizontal-edge detection, or
8. diagonal edge detection, or
9. Laplacian: a second order derivative to find edges (maxima and minima), or
10. it has not effect.

In [13]:
%use sos
# Assign your answer to this variable
answer = 5
# YOUR CODE HERE

In [14]:
%use sos
# Sanity check
if not answer in list(range(1,11)):
    print('WARNING!\nAnswer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10.')

## 2.E. Mask E
[Back to table of contents](#ToC_1_Filtering)

In the cell below, **For 0.5 points**, implement the function `filter_E`, which convolves the input image with the filter 

```python3
    [0, 0, 0, 0, 0]
    [0, 0, 0, 0, 0]
E = [0, 0, 1, 0, 0]
    [0, 0, 0, 0, 0]
    [0, 0, 0, 0, 0]
```
Remember to check if the filter is separable, and in that case, implement it in that way.

In [40]:
%use sos

# Function that filters img with the mask E
def filter_E(img):
    output = img.copy()
    
    # Filter the image with mask E
    mask_x = np.array([0, 0, 1, 0, 0])
    mask_y = np.array([0, -0, 1, 0, 0])
    
    # filter the columns
    filtered_img = ndi.convolve1d(output, mask_y, axis=0)
    # filter the rows
    filtered_img = ndi.convolve1d(filtered_img, mask_x, axis=1)
    return filtered_img

Run the next cell to visualize the result of applying `filter_E` to the image `bikesgray`.

In [41]:
%use sos
# Apply the function
bikesgray_E = filter_E(bikesgray)
# Visualize the result
plt.close('all')
vis_E = viewer([bikesgray, bikesgray_E], title=['Original', 'Filtered with mask E'], subplots=(1,2))

Button(description='Show Widgets', style=ButtonStyle())

**For 0.5 points** answer the following MCQ:

* Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring, or
2. vertical blurring, or
3. horizontal blurring, or
4. diagonal blurring, or
5. sharpening, or
6. vertical-edge detection, or
7. horizontal-edge detection, or
8. diagonal edge detection, or
9. Laplacian: a second order derivative to find edges (maxima and minima), or
10. it has not effect.

In [17]:
%use sos
# Assign your answer to this variable
answer = 10
# YOUR CODE HERE

In [18]:
%use sos
# Sanity check
if not answer in list(range(1,11)):
    print('WARNING!\nAnswer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10.')

## 2.F. Mask F
[Back to table of contents](#ToC_1_Filtering)

In the cell below, **For 0.5 points**, implement the function `filter_F`, which convolves the input image with the filter 

```python3
    [ 0, -1, -1,  0,  0]
    [-1, -1,  0,  0,  0]
F = [-1,  0,  0,  0,  1]
    [ 0,  0,  0,  1,  1]
    [ 0,  0,  1,  1,  0]
```

Remember to check if the filter is separable, and in that case, implement it in that way.

In [19]:
%use sos

# Function that filters img with the mask F
def filter_F(img):
    output = img.copy()
    
    # Filter the image with mask F
    
    mask_F = np.array([[ 0, -1, -1,  0,  0],
                       [-1, -1,  0,  0,  0],
                       [-1,  0,  0,  0,  1],
                       [ 0,  0,  0,  1,  1],
                       [ 0,  0,  1,  1,  0]])
    
    # Filter the image with mask F
    output = ndi.convolve(img, mask_F, output=np.int32)
    
    return output

Run the next cell to visualize the result of applying `filter_F` to the image `bikesgray`.

In [20]:
%use sos
# Apply the function
bikesgray_F = filter_F(bikesgray)
# Visualize the result
plt.close('all')
vis_F = viewer([bikesgray, bikesgray_F], title=['Original', 'Filtered with mask F'], subplots=(1,2))

Button(description='Show Widgets', style=ButtonStyle())

**For 0.5 points** answer the following MCQ:

* Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring, or
2. vertical blurring, or
3. horizontal blurring, or
4. diagonal blurring, or
5. sharpening, or
6. vertical-edge detection, or
7. horizontal-edge detection, or
8. diagonal edge detection, or
9. Laplacian: a second order derivative to find edges (maxima and minima), or
10. it has not effect.

In [21]:
%use sos
# Assign your answer to this variable
answer = 8
# YOUR CODE HERE

In [22]:
%use sos
# Sanity check
if not answer in list(range(1,11)):
    print('WARNING!\nAnswer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10.')

<div class="alert alert-success">
    
<p><b>Congratulations on finishing the first part of the digital filtering lab!</b></p>
<p>
Make sure to save your notebook (you might want to keep a copy on your personal computer) and upload it to <a href="https://moodle.epfl.ch/mod/assign/view.php?id=1111434">Moodle</a>, in a zip file together with the second part of this lab.
</p>
</div>

* Keep the name of the notebook as: *1_Filtering.ipynb*,
* Name the zip file: *Filtering_lab.zip*.